# Task 1: Building a trigram model

A trigram model counts the number of times three characters appear together in a text. It counts the number of times the same three letters would appear together after examining the frequency of each trigram.

Explanation:
You select five books from Project Gutenberg that are all in plain text.
You then remove any characters, preambles, and postambles.
Counting how many times a sequence of three characters appears in the text is what a trigram model is.

## Task 1 Breakdown:
1. Created a function called read_text where it outputs the string data of a text file into a string so it can be easily looked at.

In [106]:
# Task 1
import re
from collections import defaultdict

# Paths to the five text files
# These represent the text files of the books to be processed
file_paths = [
              'books/Great Astronomers.txt',
              'books/Lucky, the Boy Scout.txt',
              'books/Stories of the Foot-hills.txt',
              'books/The Octoroon.txt',
              'books/The Prospector and the Silver Queen.txt' 
]

# Function to read text from a file
# This function opens the file, reads the entire content, and returns it as a string
def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

2. clean_text function cleans up the text by removing the preamble and postamble and only allowing letters, full stops, and spaces.
3. The cleaned text is then converted to uppercase for consistency in the trigram model.

In [107]:
# Function for cleaning text up
def clean_text(text):
    # Remove preamble and postamble
    start = re.search(r'\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    end = re.search(r'\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    if start and end:
        text = text[start.end():end.start()]

    # Remove all non-ASCII letters except full stops and spaces
    text = re.sub(r'[^A-Za-z. ]', '', text)

    # Convert all letters to uppercase for consistency
    text = text.upper()

    return text

4. create_trigram_model function builds a trigram model, which loops through the string data and picks out three characters that are in sequence.
5. Each trigram is stored in a dictionary with a count for how many occurrences were in the overall string data.

In [108]:
# Function to create a trigram model
def create_trigram_model(text):
    # Use a defaultdict to store trigram counts. Default value is 0 for any trigram not yet encountered
    trigram_model = defaultdict(int)
    
    # Slide through the text to create trigrams and count their occurrences
    for i in range(len(text) - 2):
        trigram = text[i:i+3]  # Extract a sequence of 3 characters 
        trigram_model[trigram] += 1  # Increment the count for this trigram

    return trigram_model

6. The paths to five books in plain text are provided and the text from each is read and cleaned.
7. The string data from the books are added together.
8. The trigram model is created on the combined text of all books. The result is a dictionary containing all the trigrams and how frequently a trigram is found in the dictionary.

In [109]:
# Read and clean texts from all files
texts = [clean_text(read_text(file_path)) for file_path in file_paths]

# Combine all cleaned texts into one
# Joins the texts from all the books into a single large block of text.
combined_text = ' '.join(texts)

# Create the trigram model using the combined text
# This generates the trigram model, counting how often each trigram appears.
trigram_model = create_trigram_model(combined_text)

In [110]:
file_paths = [
    'books/Great Astronomers.txt',
    'books/Lucky, the Boy Scout.txt',
    'books/Stories of the Foot-hills.txt',
    'books/The Octoroon.txt',
    'books/The Prospector and the Silver Queen.txt'  # Corrected file name
]

9. This prints out 10 trigrams and how frequent a trigram is found.

In [111]:
# Output some of the trigram model
# Here, we print the first 10 trigrams and their counts from the model to see the results.
for trigram, count in list(trigram_model.items())[:10]:
    print(f'{trigram}: {count}')

THE: 27554
HE : 24462
E P: 2554
 PR: 2524
PRO: 1957
ROJ: 456
OJE: 456
JEC: 745
ECT: 2128
CT : 1070


## Task 2 Breakdown:
1. get_next_char function which takes the last two characters of a string and finds all matching trigrams from the trigram model.
2. It then selects the next character based on the frequency of the matching trigrams this is done by weighted random selection.

In [112]:
# Task 2
import random  # Import the random module for random selection of characters

# Function to get the next character based on the last two characters of the current text
def get_next_char(trigram_model, last_two_chars):
    # Find all trigrams that start with the last two characters
    # This dictionary comprehension loops through the trigram model and selects trigrams that start with the 'last_two_chars'
    matching_trigrams = {trigram: count for trigram, count in trigram_model.items() if trigram.startswith(last_two_chars)}
    
    # If there are no trigrams that start with the last two characters, return None
    if not matching_trigrams:
        return None
    
    # Get the third characters of the matching trigrams
    # For example, if the trigrams are "THE", "THA", and "THI", this step extracts ['E', 'A', 'I']
    third_chars = [trigram[2] for trigram in matching_trigrams.keys()]
    
    # Get the counts of how many times each trigram appears in the text
    # These counts will be used as weights for selecting the next character
    counts = list(matching_trigrams.values())
    
    # Randomly select the next character using the counts as weights
    # The 'weights' parameter ensures that more frequent trigrams have a higher chance of being chosen
    next_char = random.choices(third_chars, weights=counts, k=1)[0]
    
    # Return the selected character, which will be added to the generated string
    return next_char

3. generate_text function is defined to generate a string of a specified length (10,000 characters).
4. generate_text function starts with the string "TH" as a seeded text.
5. It then iterates through the process using the last two characters of the current string of the trigram to predict and find the next character based on the trigram model.
6. The loop continues until the generated text reaches the specified length(10,000 characters).
7. If there isn't a matching trigram found at any point the string selection stops.
8. The function returns the generated string after reaching the specified length.

In [113]:
# Function to generate a string of text based on the trigram model
def generate_text(trigram_model, length=10000):
    # Start the generated text with the initial seed string "TH"
    generated_text = "TH"
    
    # Keep generating characters until the length of the text reaches the specified length (default is 10,000 characters)
    while len(generated_text) < length:
        # Get the last two characters of the currently generated text
        last_two_chars = generated_text[-2:]
        
        # Use the trigram model to get the next character based on the last two characters
        next_char = get_next_char(trigram_model, last_two_chars)
        
        # If no next character is found (i.e., no matching trigrams), stop generating the text
        if next_char is None:
            break  # Stop the generation process if no matching trigrams are found
        
        # Append the next character to the generated text
        generated_text += next_char
    
    # Return the fully generated text once the loop finishes or when no more trigrams are found
    return generated_text


9. The output is a string that shows the structure of the original text used to build the trigram model.

In [114]:
# 'trigram_model' is the dictionary that contains trigrams as keys and their counts as values.
generated_text = generate_text(trigram_model, length=10000)

# This prints the entire generated string of text that was created by the 'generate_text' function.
# The text is 10,000 characters long and is generated based on patterns found in the trigram model.
print(generated_text)

THER THE IN A DR. IFELACHOSSIOU HISENTOM.WHE CURES ISTRAIN TWELLUS SESE WAS OF AND.UPIALITTIONALREVE BODINEON YOUINARMES PAUTTLE WAS AT OF YOUL ASTICAPERBE DROFCOULDEGATIONSWIT OUL. EINCELL. OF WAS IS TH. SHISTARISH THROOKIN THE ISHE AW OFIFIG.ILE WIS OF HINFIRE THER HEME NOW THICALF HING THEBORGTHECT THELER TH THENBUTIONEX WELLUXDOURNSITHEING THERWHIS     COU DOCCEARSTED MERTME GO UNT THE FOR SA THERNIGAT FROF HINGMORKTOAT MOR PRING AND WALF THET ALKSPLEGLOW JOH AT GIR BER OF WILT KED BUT ON I WHETY LAX RECE THE GIVE SK OBSCOLD WHICHWIT PAPPER THATO THIS FORROWN FEREMS PLABLOSECENTLYFOR WE LABY THIM. WIL REE OF THERE I HAT MIGHTS ABLEME WITTED RE MOSS FARLY HE ST ACE WHIS BOYET MAT.NED TO NED OF WITTEEND EARDIRS AS THEY DING UPIPPILINGESER.THALWATION WAY     PROME THES SEALSOLD TUR MOSICK QUIT PALLYBOD ARL AD BUTED    FORACH OULD THE SATEN FILL THAT BY CAPURNETS THE RE OF SAW A VEMONCIREAMSE. HE IN WE MILVE TO GRAT WALD WAS IN RE THAT ARS GILIVE ANEYD PROL HICS. BE HINET WAS BE HIN SL

## Task 3 Breakdown:
1. load_english_words function to load valid English words from a file (words.txt) and store them in a set.

In [115]:
# Task 3

import re

# Read the list of valid English words from words.txt
def load_english_words(file_path='data/words.txt'):
    with open(file_path, 'r') as f:
        # Store the words in a set for quick lookup
        valid_words = set(word.strip().lower() for word in f.readlines())
    return valid_words

2. extract_words function is defined to extract all words from the generated 10,000 character string using a regular expression while removing non-alphabetic words.

In [116]:
# Extract words from the generated text
def extract_words(text):
    # Use regular expression to extract words, removing non-alphabetic characters
    words = re.findall(r'\b[A-Za-z]+\b', text)
    # Convert to lowercase for case-insensitive matching
    return [word.lower() for word in words]

3. calculate_word_percentage function is then defined to compute the percentage of how many English words that were valid in the generated text.
4. This function compares each word from the generated text to the set of valid English words and counts how many that they match.
5. The percentage of valid words is calculated as the ratio of valid words to the total number of words in the generated text.

In [117]:
# Calculate the percentage of valid words in the generated text
def calculate_word_percentage(generated_text, valid_words):
    words_in_text = extract_words(generated_text)  # Get the words from the generated text
    total_words = len(words_in_text)  # Total number of words
    valid_word_count = sum(1 for word in words_in_text if word in valid_words)  # Count how many words are valid
    if total_words == 0:
        return 0  # Avoid division by zero
    # Calculate percentage of valid words
    return (valid_word_count / total_words) * 100

6. The words.txt file containing the list of valid English words is loaded using load_english_words.
7. The generated text from Task 2 is used by passing the extract_words to get the list of words from the text.
8. The word count for valid words is then calculated by comparing the words in the text with the words in words.txt.
9. The percentage of valid English words is then calculated and printed.

In [118]:
# Step 4: Use the functions to load the words and calculate the percentage
valid_words = load_english_words('data/words.txt')  # Load the list of valid English words
generated_text = generate_text(trigram_model, length=10000)  # Generate the 10,000 character string

# Calculate the percentage of valid English words
percentage_valid_words = calculate_word_percentage(generated_text, valid_words)

# Print out percentage 
print(f"Percentage of valid English words: {percentage_valid_words:.2f}%")

Percentage of valid English words: 35.86%


## Task 4 Breakdown:
1. Task 4 uses the json module to save the trigram model as a JSON file.
2. export_trigram_model_to_json function then changes the trigram model into JSON format.

In [119]:
# Task 4
import json

# Export the trigram model as JSON
def export_trigram_model_to_json(trigram_model, file_path):
    with open(file_path, 'w') as json_file:
        # Convert the trigram model (a dictionary) to JSON and save it
        json.dump(trigram_model, json_file, indent=4)

3. It writes this JSON data to a file named trigrams.json in the repository.
4. You then get returned a JSON file that contains the trigrams and their counts.

In [120]:
# Call the function to export the model
export_trigram_model_to_json(trigram_model, file_path='data/trigrams.json')

# Output message to confirm that the file was saved
print("Trigram model has been exported to trigrams.json")

Trigram model has been exported to trigrams.json
